In [1]:
# Let's build GPT: from scratch, in code, spelled out.(https://youtu.be/kCc8FmEb1nY?si=YTkDSOYk5S_CJVtz)

In [2]:
with open('tinyshakespeare.txt', 'r') as f:
    text = f.read()

In [3]:
print(f'sample: \n{text[:100]}\n\ntotal text lenght : {len(text)}')

sample: 
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

total text lenght : 1115394


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode =lambda s: [stoi[c] for c in s]
decode =lambda l: ''.join([itos[i] for i in l])
print(encode('hi there!'))
print(decode(encode('hi there!')))

[46, 47, 1, 58, 46, 43, 56, 43, 2]
hi there!


In [6]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(f'sample : \n{data[:100]}\n\nshape = {data.shape}, dtype = {data.dtype}')

sample : 
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

shape = torch.Size([1115394]), dtype = torch.int64


In [7]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context, '--->', target)

tensor([18]) ---> tensor(47)
tensor([18, 47]) ---> tensor(56)
tensor([18, 47, 56]) ---> tensor(57)
tensor([18, 47, 56, 57]) ---> tensor(58)
tensor([18, 47, 56, 57, 58]) ---> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) ---> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) ---> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) ---> tensor(58)


In [9]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = {
        'train' : train_data,
        'val' : val_data
    }[split]
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print(f'input :\n{xb.shape}\n{xb}\ntarget :\n{yb.shape}\n{yb}')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b,t]
        print(context, '-->', target)

input :
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target :
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
tensor([24]) --> tensor(43)
tensor([24, 43]) --> tensor(58)
tensor([24, 43, 58]) --> tensor(5)
tensor([24, 43, 58,  5]) --> tensor(57)
tensor([24, 43, 58,  5, 57]) --> tensor(1)
tensor([24, 43, 58,  5, 57,  1]) --> tensor(46)
tensor([24, 43, 58,  5, 57,  1, 46]) --> tensor(43)
tensor([24, 43, 58,  5, 57,  1, 46, 43]) --> tensor(39)
tensor([44]) --> tensor(53)
tensor([44, 53]) --> tensor(56)
tensor([44, 53, 56]) --> tensor(1)
tensor([44, 53, 56,  1]) --> tensor(58)
tensor([44, 53, 56,  1, 58]) --> tensor(46)
tensor([44, 53, 56,  1, 58, 46]) --> tensor(39)
tensor([44, 53, 56,  1, 58, 46, 39]) --> tenso

In [10]:
# bigram

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx) # idx(B = batch,T = time,C = channel)
        
        if targets is not None :
            B, T, C = logits.shape # we don't change the shape if we don't have to calculate loss
            logits = logits.view(B * T, C)
            targets = targets.view(B * T) # reshape the input and the target as pytorch assumes B,C,T inputs for -log loss/cross entropy

        loss = None
        if targets is not None :
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        #idx is B, T in the current context
        for _ in range(max_new_tokens):
            
            # prediction
            logits, loss = self(idx)
            
            # get last time prediction
            logits = logits[:, -1, :] #(B, C)
            probs = F.softmax(logits, dim = 1)
            idx_next = torch.multinomial(probs, num_samples = 1) #(B, T = 1)
            
            #append to result
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T + 1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item()) # this is a bigram model, best scenario is -ln(1/65) -ln(1/vocab_size)

context = torch.zeros((1, 1), dtype = torch.long) # start of prediction (C = ' ', T = 0)
print(decode(m.generate(idx = context, max_new_tokens = 100)[0].tolist())) # we work on batches so we have to take the first one as we had only one as input

torch.Size([32, 65])
4.878634929656982

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [11]:
#optimize bigram
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [17]:
batch_size = 32
for step in range(10_000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = None)
    loss.backward()
    optimizer.step()
print(loss.item())

2.4988932609558105


In [18]:
context = torch.zeros((1, 1), dtype = torch.long) # start of prediction (C = ' ', T = 0)
print(decode(m.generate(idx = context, max_new_tokens = 100)[0].tolist())) # we work on batches so we have to take the first one as we had only one as input


g h be y cou prornon
A urelann, dwowoeaus m no o this?

Whechand y CHanted ar dinadgha os!
ABut:
Sti
